In [1]:
!pip install deap update_checker tqdm stopit tpot

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 4.4 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=99d1b9422b6d8262abef004a54226665eef38a2d308370c8235fa0c92b7dd976
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.

In [3]:
import pandas as pd
import numpy as np
from tpot import TPOTRegressor


df_test=pd.read_csv("TestSet_regression.csv")
df_train=pd.read_csv("TrainSet_regression.csv")

X_train = df_train.drop(["Price"], axis=1).values  # TPOT needs 2D-Array
y_train = df_train["Price"].values  #price must be 1D

X_test = df_test.drop(["Price"], axis=1).values
y_test = df_test["Price"].values




In [6]:
# initializing tpot with parameters
#max_time_mins=None (per default) - here max time 4h
#config_dic due to one hot encoding
tpot = TPOTRegressor(generations=5, population_size=50,
                     cv=5, verbosity=2, n_jobs=-1, max_time_mins=240,
                     periodic_checkpoint_folder='/content/results')

In [7]:
#starting the training
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/5 [00:00<?, ?pipeline/s]


2.91 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RidgeCV(input_matrix)


TPOTRegressor(cv=2, generations=2, max_time_mins=2, n_jobs=-1,
              periodic_checkpoint_folder='/content/results', population_size=5,
              verbosity=2)

In [8]:
import sklearn.metrics as skm

y_pred = tpot.predict(X_test)

print("RESULTS OF BEST MODEL:\n")

print(f"R2:                         {skm.r2_score(y_test, y_pred)}")
print(f"Mean Squared Error:         {skm.mean_squared_error(y_test, y_pred)}")
print(f"Root Mean Squared Error:    {skm.mean_squared_error(y_test, y_pred, squared=False)}")
print(f"Mean Absolute Error:        {skm.mean_absolute_error(y_test, y_pred)}")
print(f"Explained Variance:         {skm.explained_variance_score(y_test, y_pred)}")
print(f"Max Error:                  {skm.max_error(y_test, y_pred)}")
print(f"Median Absolute Error       {skm.median_absolute_error(y_test, y_pred)}")


RESULTS OF BEST MODEL:

R2:                         0.2788144648890476
Mean Squared Error:         2881647256867.2935
Root Mean Squared Error:    1697541.5331788773
Mean Absolute Error:        1330234.0607542272
Explained Variance:         0.28008968897596287
Max Error:                  6948730.018056526
Median Absolute Error       1107789.2391786822


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
from google.colab import files
# output of values to dictionary
metrics_dict = {
    'Metric': [
        'R2',
        'Mean Squared Error',
        'Root Mean Squared Error',
        'Mean Absolute Error',
        'Explained Variance',
        'Max Error',
        'Median Absolute Error'
    ],
    'Value': [
        skm.r2_score(y_test, y_pred),
        skm.mean_squared_error(y_test, y_pred),
        skm.mean_squared_error(y_test, y_pred, squared=False),
        skm.mean_absolute_error(y_test, y_pred),
        skm.explained_variance_score(y_test, y_pred),
        skm.max_error(y_test, y_pred),
        skm.median_absolute_error(y_test, y_pred)
    ]
}

#to df
metrics_df = pd.DataFrame(metrics_dict)


print(metrics_df)

metrics_df.to_csv('tpot_regression_metrics_normal.csv', index=False)
files.download('tpot_regression_metrics_normal.csv')

                    Metric         Value
0                       R2  4.600869e-01
1       Mean Squared Error  2.953344e+12
2  Root Mean Squared Error  1.718530e+06
3      Mean Absolute Error  9.542693e+05
4       Explained Variance  4.602554e-01
5                Max Error  3.825606e+07
6    Median Absolute Error  5.718749e+05


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tpot.export('tpot_regression_model_normal')